In [1]:
import requests

In [71]:
header = {
  "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
  "Referer": "https://translate.kakao.com/"
}

def translate_to_jp(text):
  data = {
    "queryLanguage": "kr",
    "resultLanguage": "jp",
    "q": text
  }
  k = requests.post("https://translate.kakao.com/translator/translate.json", data=data, headers=header)
  return " ".join([out for array in k.json()["result"]["output"] for out in array])

def translate_to_kr(text):
  data = {
    "queryLanguage": "jp",
    "resultLanguage": "kr",
    "q": text
  }
  k = requests.post("https://translate.kakao.com/translator/translate.json", data=data, headers=header)
  return " ".join([out for array in k.json()["result"]["output"] for out in array])

In [108]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium").cuda()

In [123]:
def generate(text, **kwargs):
  inputs = tokenizer(translate_to_jp(text), add_special_tokens=False, return_tensors="pt")["input_ids"]
  output = model.generate(inputs.cuda(), **kwargs).cpu().numpy()[0]
  return translate_to_kr(tokenizer.decode(output))

In [127]:
한국어입력 = "근육을 키우기 위해서는 "
generate(한국어입력, do_sample=True, max_length=100, top_k=50, top_p=0.95, repetition_penalty=2.0, early_stpping=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"근육을 키우기 위해 훈련을 시작한 지 2~3개월이 지난 무렵부터 효능이 있는 것 같습니다. 근육 트레이가 오래 지속되기 어려운 원인 중 하나로는 ‘질리기 쉽다’는 것도 있네요 (웃음) 이 '동기유지방법', '계속하는 요령' '방식대로 개선·향상시키는 것이 가능한 것들'에 대해 소개해 봅니다! “처음에 확인해두고 싶은 게 뭐지?”라고 생각한 분도 있을지도 모릅니다."